# ***Proyecto EDA: Airbnb*** 
## ***Limpieza y Transformación***

**Autor:** Óscar Fernández-Chinchilla López

![Logo Airbnb](../images/airbnb-logo.png)

# 📋 ***Índice***

1. [Introducción y objetivos de la limpieza](#introduccion)
2. [Carga del dataset original e importación de las librerias](#carga)
3. [Normalizaxión de las columnas](#normalizacion)
4. [Proceso de limpieza](#proceso)
   - [Limpieza de columnas monetarias](#monetarias)
   - [Limpieza de columnas categóricas](#categoricas)
   - [Formateo de fechas](#fechas)
   - [Transformación de columnas numéricas (Object) a Integer o Float](#numericas)
5. [Eliminación de duplicados](#duplicados)
6. [Tratamiento de rangos irreales (outliers imposibles para Airbnb)](#rangos)
7. [Comparativas finales conforme al inicio de la limpieza](#comparativas)
8. [Últimas comprobaciones de consistencias en las reseñas](#consistencia)
9. [Exportación a CSV](#exportacion)


<a id="introduccion"></a>
## 1️⃣ ***Introducción y objetivos de la limpieza***


Para poder realizar un análisis fiable del dataset y extraer conclusiones realmente
representativas, es necesario corregir los problemas identificados durante la fase de
exploración. Por ello, en este notebook vamos a llevar a cabo un proceso de limpieza y
transformación del dataset.

## 🎯 Objetivos de la limpieza:

- **Renombrar y limpiar** los nombres de las columnas para facilitar su manejo.
- **Eliminar** duplicados que puedan generar ruido o sesgos en el análisis.
- **Tratar** los valores nulos, tomando decisiones justificadas en cada caso:
  - **Eliminar** columnas o filas con demasiados nulos o información irrelevante
  - **Imputar** valores cuando la información sea importante mantenerla
- **Corregir** los tipos de datos que no sean coherentes (ej.: `price` como `object` → `float`)
- **Estandarizar** categorías con errores ortográficos o incoherencias (ej.: boroughs mal escritos)
- **Detectar y ajustar outliers** para evitar que afecten de forma negativa a análisis posteriores
- **Formatear** las fechas correctamente para futuros cálculos temporales

---

Una vez completado este proceso, obtendremos un dataset **limpio y preparado** para continuar con
el análisis y la visualización avanzada.



<a id="carga"></a>
# 2️⃣ ***Carga del dataset original e importación de librerías***

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os

sys.path.append(os.path.abspath(".."))

# Cargamos el dataset original
airbnb_df = pd.read_csv("../data/Airbnb_Open_Data.csv", low_memory=False)

# Creamos una copia para trabajar con ella y mantener el original intacto en caso de error
airbnb_clean = airbnb_df.copy()

<a id="normalizacion"></a>
# 3️⃣ ***Normalización de las columnas***

## Para iniciar el proceso de limpieza, comenzaremos normalizando los nombres de las columnas del dataset.
Este paso es fundamental, ya que permite un acceso más eficiente a las variables y evita problemas derivados de inconsistencias como espacios en blanco, símbolos no deseados o diferencias en el uso de mayúsculas y minúsculas, entre otros posibles errores tipográficos.

In [3]:
from src.cleaning import normalize_column_names
# Una vez tenemos la función creada, la utilizamos con la copia del dataset original para poder empezar a trabajar
airbnb_clean = normalize_column_names(airbnb_clean)
airbnb_clean.columns

Index(['id', 'name', 'host_id', 'host_identity_verified', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'lat', 'long', 'country',
       'country_code', 'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'price', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'review_rate_number', 'calculated_host_listings_count',
       'availability_365', 'house_rules', 'license'],
      dtype='object')

#### Como se puede observar en el anterior output, todos los nombres de las columnas del dataset aparecen en minúscula, sin espacios en los extremos y con separadores de "_" entre palabras.

<a id="proceso"></a>
# 4️⃣ ***Proceso de limpieza***

#### En este apartado nos centraremos en la limpieza y transformación de las columnas del dataset que requieren ser estandarizadas, imputadas o convertidas a los tipos de datos adecuados. Esto permitirá garantizar la coherencia y correcta interpretación de la información para los análisis y cálculos posteriores.

<a id="monetarias"></a>
## ***4.1 Limpieza de columnas monetarias***

#### En este apartado decido ***limpiar*** primero los datos de la columna de ***Precio*** y ***Tarifa*** de Servicio para eventualmente en el apartado de análisis poder correlacionar correctamente las variables de estas columnas y poder realizar cálculos matemáticos. 
#### Una vez los datos han sido limpiados, se convierten a tipo Float para poder utilizarlos en el futuo, los que no se puedan convertir se dejarán como NaN para no perder información por el camino.

In [4]:
from src.cleaning import clean_monetary_columns
# Creo la lista con los nombres de las columnas a limpiar
monetary_columns = ["price", "service_fee"]
airbnb_clean = clean_monetary_columns(airbnb_clean,monetary_columns)

#### Con las siguientes celdas comprobamos si la limpieza y la transformación de las columnas se ha realizado correctamente:

In [5]:
airbnb_clean.price.info

<bound method Series.info of 0          966.0
1          142.0
2          620.0
3          368.0
4          204.0
           ...  
102594     844.0
102595     837.0
102596     988.0
102597     546.0
102598    1032.0
Name: price, Length: 102599, dtype: float64>

In [6]:
airbnb_clean.service_fee.info

<bound method Series.info of 0         193.0
1          28.0
2         124.0
3          74.0
4          41.0
          ...  
102594    169.0
102595    167.0
102596    198.0
102597    109.0
102598    206.0
Name: service_fee, Length: 102599, dtype: float64>

<a id="categoricas"></a>
## ***4.2 Limpieza de columnas categóricas***

#### En el siguiente apartado utilizamos una función para limpiar y transformar las columnas categoricas seleccionadas y observadas previamente en el Notebook de exploración.
#### En cuanto a la columna de los barrios pequeños del dataset, decido limpiarlos porque algunos valores contienen más de 1 palabra separadas por comas, con valores erroneos de las variables. Ejemplo: "Chealse, Staten Island". Chealse no forma parte del Borough de Staten Island, por lo que decido quedarme con la parte de la izquierda de la coma y descartar la parte posterior.

In [7]:
# Importamos una función que transforma las columnas categóricas de tipo Object a Categorical
from src.cleaning import clean_categorical_values
airbnb_clean = clean_categorical_values(airbnb_clean)

In [8]:
# Comprobamos si los tipos han sido cambiados a categóricos
airbnb_clean.dtypes

id                                   int64
name                                object
host_id                              int64
host_identity_verified            category
host_name                           object
neighbourhood_group               category
neighbourhood                     category
lat                                float64
long                               float64
country                           category
country_code                      category
instant_bookable                   boolean
cancellation_policy               category
room_type                         category
construction_year                  float64
price                              float64
service_fee                        float64
minimum_nights                     float64
number_of_reviews                  float64
last_review                         object
reviews_per_month                  float64
review_rate_number                 float64
calculated_host_listings_count     float64
availabilit

<a id="fechas"></a>
## ***4.3 Formateo de las fechas de ultima review (por anuncio)***

#### En el siguiente apartado se formatearán las fechas de la columna "last_review" para que obtenga un formato datetime y asi porder acceder a valores como el año, mes y dia en el análisis.
#### Las fechas que no se pueden transformar porque son nulas, he decidido mantenerlas porque al eliminarlas podría perderse información importante por el camino, que luego nos podria resultar muy útil al analizar otros valores del dataset.

In [18]:
from src.cleaning import format_dates
airbnb_clean = format_dates(airbnb_clean)
df = airbnb_clean["last_review"]
df

0        2021-10-19
1        2022-05-21
2               NaT
3        2019-07-05
4        2018-11-19
            ...    
102053   2019-03-27
102054   2017-08-31
102055   2019-06-26
102056          NaT
102057   2019-06-15
Name: last_review, Length: 102058, dtype: datetime64[ns]

####  Si un anuncio no tiene una fecha de ultima reseña, puede ser porque sea un anuncio nuevo. Por eso es útil mantener los valores nulos.

<a id="numericas"></a>
## ***4.4 Transformación de columnas numéricas a Float o Integer***

#### En este apartado utilizamos una función que recogerá por parametro el dataframe, y las columnas a convertir Integer o Float según la intención que tengamos para analizarlas.

### Justificación de las transformaciones:
- Número de reviews: Considero que es más importante conservarla como Integer para posteriormente realizar Countplots y análisis bivariados.
- Noches mínimas: No tiene sentido que el tipo sea Float ya que debe se un número entero.
- Conteo de anuncios por anfritión: Por el mismo motivo que la anterior, tiene mas coherencia que sea de tipo Integer. Esto permitirá saber si un anfitrión realmente es una empresa o un particular en base a su número de anuncios simultáneos.
- Disponibilidad: La dispponibilidad de dias al año no puede ser un decimal.
- Año de construcción: Al dejarlo como Float podría dar pie a errores en los análisis gráficos, por lo que prefiero convertirlo a Integer.
- Reviews al mes de un anuncio: Tiene sentido que sea Float ya que la media puede ser decimal.

In [ ]:
from src.cleaning import convert_numeric_types
airbnb_clean = convert_numeric_types(
    airbnb_clean,
    int_cols=["number_of_reviews", "minimum_nights", "calculated_host_listings_count", "availability_365","construction_year"],
    float_cols=["reviews_per_month"]
)
# Muestro los tipos de las columnas alteradas
airbnb_clean[
    ["number_of_reviews", "minimum_nights", "calculated_host_listings_count", "availability_365", "construction_year","reviews_per_month","review_rate_number"]
].dtypes

number_of_reviews                   Int64
minimum_nights                      Int64
calculated_host_listings_count      Int64
availability_365                    Int64
construction_year                   Int64
reviews_per_month                 float64
review_rate_number                float64
dtype: object

<a id="duplicados"></a>
# 5️⃣ ***Eliminación de duplicados***

#### En este apartado se eliminarás las columnas completamente duplicadas, ya que considero que al dejarlas pueden generar confusión en el análisis y que además pueden tener diferentes formatos

In [ ]:
# Compruebo cuantos duplicados tengo actualmente en el dataset
airbnb_clean.duplicated().sum()

np.int64(541)

### ***Tenemos 541 filas duplicadas***

#### Ahora eliminamos del dataframe las filas duplicadas, dejando solo la primera instancia real de las columnas originales

In [11]:
airbnb_clean.drop_duplicates(inplace=True)

#### En el siguiente paso volvemos a sacar la suma de duplicado que existe en el dataframe para comprobar que se hayan eliminado correctamente

In [12]:
airbnb_clean.duplicated().sum()

np.int64(0)

### ***Resulatdo de duplicados: 0***

<a id="rangos"></a>
# 6️⃣ ***Tratamiento de los rangos irreales o sin sentido.***

### Tratamientos aplicados sobre valores erróneos en columnas numéricas

Durante el proceso de limpieza del dataset de Airbnb, se identificaron valores imposibles o incoherentes en algunas columnas numéricas que afectaban la calidad del análisis. Por ello, se aplicaron los siguientes tratamientos:

---

#### `availability_365`
Esta columna indica el número de días al año que el alojamiento está disponible.  
Cualquier valor **fuera del rango 0–365** se considera erróneo:

- Valores **> 365** → imposibles (un año no tiene más días)
- Valores **< 0** → incoherentes (no existe disponibilidad negativa)

In [ ]:
# availability_365 no puede ser > 365
airbnb_clean.loc[airbnb_clean["availability_365"] > 365, "availability_365"] = pd.NA
airbnb_clean.loc[airbnb_clean["availability_365"] < 0, "availability_365"] = pd.NA

#### `minimum_nights`
Representa la estancia mínima requerida para reservar un alojamiento.

- Valores <= 0 no tienen sentido en el contexto de una reserva

In [ ]:
# minimum_nights no puede ser <= 0
airbnb_clean.loc[airbnb_clean["minimum_nights"] <= 0, "minimum_nights"] = pd.NA

#### `construction_year`
Año en el que se construyó la propiedad. Se validan los rangos temporales realistas:

- Menor a 1800 → sin fiabilidad histórica

- Mayor al año actual → propiedad inexistente o error en el dato

In [ ]:
# construction_year imposible (antes de 1800 o > año actual)
current_year = pd.Timestamp.now().year
airbnb_clean.loc[airbnb_clean["construction_year"] < 1800, "construction_year"] = pd.NA
airbnb_clean.loc[airbnb_clean["construction_year"] > current_year, "construction_year"] = pd.NA

In [ ]:
print("availability_365:", airbnb_clean["availability_365"].min(), "-", airbnb_clean["availability_365"].max())
print("minimum_nights:", airbnb_clean["minimum_nights"].min(), "-", airbnb_clean["minimum_nights"].max())
print("construction_year:", airbnb_clean["construction_year"].min(), "-", airbnb_clean["construction_year"].max())

availability_365: 0 - 365
minimum_nights: 1 - 5645
construction_year: 2003 - 2022


<a id="comparitivas"></a>
# 7️⃣ ***Comparativas finales conforme al inicio de la limpieza***

In [ ]:
# Creamos un dataframe que muestre los datos del dataset antes de ser limpiado
comparison_nulls = pd.DataFrame({
    "Antes": airbnb_df.isna().sum(),
    "Tipo de dato": airbnb_df.dtypes,
})
comparison_nulls

,Antes,Tipo de dato
id,0,int64
NAME,250,object
host id,0,int64
host_identity_verified,289,object
host name,406,object
neighbourhood group,29,object
neighbourhood,16,object
lat,8,float64
long,8,float64
country,532,object


In [ ]:
# Creamos un dataframe que muestre los datos del dataset despueés de ser limpiado
comparison_nulls = pd.DataFrame({
    "Despues": airbnb_clean.isna().sum(),
    "Tipo de dato": airbnb_clean.dtypes,
})
comparison_nulls

,Despues,Tipo de dato
id,0,int64
name,250,object
host_id,0,int64
host_identity_verified,0,category
host_name,406,object
neighbourhood_group,0,category
neighbourhood,0,category
lat,8,float64
long,8,float64
country,532,category


<a id="consistencia"></a>
# ***8️⃣ Consistencia entre reseñas, fechas y valoraciones***

En este apartado se ha revisado la coherencia interna entre las columnas relacionadas con la actividad de reseñas de los anuncios:

- `number_of_reviews`: número total de reseñas recibidas.
- `last_review`: fecha de la última reseña (tipo datetime/NaT).
- `reviews_per_month`: número de reseñas mensuales (float).
- `review_rate_number`: valoración media de las reseñas.

El objetivo es garantizar que los valores de estas columnas sean consistentes entre sí y representen correctamente la realidad del anuncio.

---

### ✅ Regla 1: Anuncios sin reseñas (`number_of_reviews = 0`)

Para los anuncios que no han recibido ninguna reseña:

- No existe fecha de última reseña (`last_review` debería ser NaT).
- No hay actividad mensual de reseñas (`reviews_per_month` debe ser 0.0).
- No tiene sentido una valoración media (`review_rate_number` debe ser nulo).

Por tanto, se fuerza esta lógica:

In [19]:

# Anuncios sin reseñas → 0 reseñas al mes
airbnb_clean.loc[
    airbnb_clean["number_of_reviews"] == 0,
    "reviews_per_month"
] = 0.0

# Anuncios sin reseñas → rating no aplicable
airbnb_clean.loc[
    airbnb_clean["number_of_reviews"].fillna(0) == 0,
    "review_rate_number"
] = pd.NA

### ✅ Regla 2: reviews_per_month nulo con reseñas existentes

En los casos donde un anuncio sí tiene reseñas (number_of_reviews > 0), pero reviews_per_month es nulo:

- No se interpreta como “0 reseñas al mes”.

- Se considera un dato faltante real (no informado o no calculado).

- Se mantiene NaN para permitir un mejor analisis.

In [20]:
# Identificar casos con inconsistencia: reseñas > 0 pero reviews_per_month nulo
inconsistentes_rpm = airbnb_clean[
    (airbnb_clean["number_of_reviews"] > 0) &
    (airbnb_clean["reviews_per_month"].isna())
]

### ✅ Regla 3: Valoración media sin reseñas (review_rate_number con number_of_reviews = 0 o NaN)

La columna review_rate_number solo tiene sentido si existe al menos una reseña asociada.
Si un anuncio no tiene reseñas (number_of_reviews = 0) o se desconoce cuántas tiene, cualquier valor en review_rate_number se considera incoherente.

En estos casos se elimina la valoración:

In [21]:
airbnb_clean.loc[
    airbnb_clean["number_of_reviews"].fillna(0) == 0,
    "review_rate_number"
] = pd.NA

### ***Comparación posterior al último tratamiento:***

In [22]:
comparison_nulls = pd.DataFrame({
    "Despues": airbnb_clean.isna().sum(),
    "Tipo de dato": airbnb_clean.dtypes,
})
comparison_nulls

,Despues,Tipo de dato
id,0,int64
name,250,object
host_id,0,int64
host_identity_verified,0,category
host_name,406,object
neighbourhood_group,0,category
neighbourhood,0,category
lat,8,float64
long,8,float64
country,532,category


<a id="exportacion"></a>
# 9️⃣ ***Exportación del dataset limpio a CSV***

In [24]:
# En esta celda exportamos el dataset ya limpiado y tratado para seguir con la parte de visualización
airbnb_clean.to_csv("../data/airbnb_clean.csv", index=False)
inconsistentes_rpm.to_csv("../data/airbnb_inconsistencias.csv", index=False)